# Importing Packages

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Creating read_csv Function

In [ ]:
def read_my_data(csv_file):
    my_data = pd.read_csv(csv_file)
    return my_data

In [ ]:
my_data = read_my_data("python_assignment_1.csv")

# Tidying Data

In [ ]:
my_data.columns = my_data.columns.str.title()
my_data = my_data.replace(['positiveprime', 'negativeprime', 'positivetarget', 'negativetarget'], 
                ['Positive', 'Negative', 'Positive', 'Negative'])

In [ ]:
my_data.head()

In [ ]:
def run_my_anova(IV1, IV2, DV, data):
    mapping = {data.columns[0]: 'Participant', data.columns[1]: IV1, data.columns[2]: IV2, data.columns[3]: DV}
    data_frame = data.rename(columns=mapping)
    descriptives = data_frame[[IV1, IV2, DV]].groupby([IV1,IV2]).describe()
    return descriptives

In [ ]:
run_my_anova('Prime', 'Target', 'RT', my_data)

# Going beyond

## Defining Columns

In [ ]:
IV1 = input('Enter your first independent variable: ')
IV2 = input('Enter your second independent variable: ')
DV = input('Enter your dependent variable: ')
mapping = {my_data.columns[1]: IV1, my_data.columns[2]: IV2, my_data.columns[3]: DV}
data = my_data.rename(columns=mapping)
data.columns = data.columns.str.title()
data

## Tidying data

## Descriptive stats

In [ ]:
carry_out_summary = input('Would you like to generate some summary statistics? y/n: ')
if carry_out_summary == 'y':
    data_frame[[IV1, IV2, DV]].groupby([IV1,IV2]).describe()

## Adding options

## Defining functions

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
def anova_function():
    global data
    options = input('''
What would you like to do with your data:
Enter 1 to load in the data file
Enter 2 to view data frame
Enter 3 to tidy data
Enter 4 to generate summary statistics
Enter 5 to visualise the data
Enter 6 to perform and report an ANOVA
Enter 7 to perform and report pairwise comparisons

''')
    if options == '1':
        substring = 'csv'
        def load_data_function():
            global data
            global IV1
            global IV2
            global DV
            file = input('''
Enter your csv file name or enter 'b' to go back: 

        ''')
            if substring in file:
                print('csv file loaded in.')
                data = pd.read_csv(file)
                IV1 = data.columns[1]
                IV2 = data.columns[2]
                DV = data.columns[3]
            elif file == 'b':
                anova_function()
            else:
                print('Invalid option. Try again.')
                load_data_function_again()
        def load_data_function_again():
            load_data_function()
        load_data_function()        
    elif options == '2':
        print('\033[1m' + 'Displaying Data Frame')
        display(data)
    elif options == '3':
        def tidy_data_function():
            global data
            global IV1
            global IV2
            global DV
            columns_levels = input('''
What would you like to tidy?
Enter 1 to rename the columns
Enter 2 to rename the levels
Enter 3 to go back

        ''')
            if columns_levels == '1':
                IV1 = input('''
Enter your first independent variable: 

            ''')
                IV2 = input('''
Enter your second independent variable: 

            ''')
                DV = input('''
Enter your dependent variable: 
            
            ''')
                mapping = {data.columns[0]: 'Participant', data.columns[1]: IV1, data.columns[2]: IV2, data.columns[3]: DV}
                data = data.rename(columns=mapping)
            elif columns_levels == '2':
                def levels_rename():
                    global data
                    level_origin = input('''
What would you like to rename: 

                ''')
                    level_change = input('''
What would you like to rename it to: 

                ''')
                    data = data.replace([level_origin],[level_change])

                    levels_rename_again()

                def levels_rename_again():
                    global data
                    change_levels = input('''
Would you like to rename another level? y/n: 

            ''')
                    if change_levels == 'y':
                        levels_rename()
                    elif change_levels == 'n':
                        anova_function()
                    else:
                        levels_rename_again()        
                levels_rename()
            elif columns_levels == '3':
                anova_function()
            else:
                print('Invalid choice. Try again')
                tidy_data_function_again()
        def tidy_data_function_again():
            tidy_data_function()
        tidy_data_function()
    elif options == '4':
        print ('\033[1m' + 'Displaying summary statistics')
        display(data[[IV1, IV2, DV]].groupby([IV1,IV2]).describe())
    elif options == '5':
        

    anova_function_again()

def anova_function_again():
    anova_function()
    
anova_function()


What would you like to do with your data:
Enter 1 to load in the data file
Enter 2 to view data frame
Enter 3 to tidy data
Enter 4 to generate summary statistics
Enter 5 to visualise the data
Enter 6 to perform and report an ANOVA
Enter 7 to perform and report pairwise comparisons

1

Enter your csv file name or enter 'b' to go back: 

        python_assignment_1.csv
csv file loaded in.

What would you like to do with your data:
Enter 1 to load in the data file
Enter 2 to view data frame
Enter 3 to tidy data
Enter 4 to generate summary statistics
Enter 5 to visualise the data
Enter 6 to perform and report an ANOVA
Enter 7 to perform and report pairwise comparisons

4
Displaying summary statistics


rt                                           \
                              count         mean        std     min      25%   
prime         target                                                           
negativeprime negativetarget  148.0  1547.256757  52.382700  1417.0  1507.50   
              positivetarget  148.0  1562.648649  50.468400  1440.0  1533.25   
positiveprime negativetarget  148.0  1566.959459  54.016737  1448.0  1528.50   
              positivetarget  148.0  1547.391892  44.879072  1430.0  1513.00   

                                                       
                                 50%      75%     max  
prime         target                                   
negativeprime negativetarget  1550.5  1580.50  1710.0  
              positivetarget  1562.0  1587.00  1736.0  
positiveprime negativetarget  1570.5  1602.25  1694.0  
              positivetarget  1550.5  1584.25  1637.0

In [ ]:
def run_my_anova(IV1, IV2, DV, data):
    mapping = {data.columns[0]: 'Participant', data.columns[1]: IV1, data.columns[2]: IV2, data.columns[3]: DV}
    data_frame = data.rename(columns=mapping)
    descriptives = data_frame[[IV1, IV2, DV]].groupby([IV1,IV2]).describe()
    return descriptives

In [ ]:
IV1 = data.columns[1]
IV2 = data.columns[2]
DV = data.columns[3]

In [ ]:
data[[IV1, IV2, DV]].groupby([IV1,IV2]).describe()